In [2]:
from PIL import Image
import numpy as np
img = np.array(Image.open("/mnt/disk1/tritd/CropFromWeb/snapshot-2024-03-05T00_00_00Z.tif"))
img.shape

(4656, 4176, 4)

In [1]:
from model.PAD_convLSTM import ConvLSTM
from pathlib import Path
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.strategies import DDPStrategy as DDPPlugin
# from pytorch_lightning.plugins import DDPPlugin
# from pytorch_lightning.strategies import DDPStrategys
import torch
from utils.settings.config import RANDOM_SEED, CROP_ENCODING, LINEAR_ENCODER, CLASS_WEIGHTS, BANDS
run_path = Path(*Path("/mnt/disk1/tritd/S4A-Models/logs/convlstm/100epoch/run_20240228195928/checkpoints/epoch=7-step=30240.ckpt").parts[:-2])
model = ConvLSTM.load_from_checkpoint("/mnt/disk1/tritd/S4A-Models/logs/convlstm/100epoch/run_20240228195928/checkpoints/epoch=7-step=30240.ckpt",
                                              map_location=torch.device('cpu'),
                                              run_path=run_path,
                                              linear_encoder=LINEAR_ENCODER,
                                              checkpoint_epoch=0)

In [9]:
numpy_array = np.random.rand(2, 1, 4, 61, 61)

# Converting numpy array to a torch FloatTensor
input = torch.FloatTensor(numpy_array).cuda()
model = model.cuda()
output = model(input)
print(output.shape)

torch.Size([2, 12, 61, 61])


In [13]:
import numpy as np

# Simulate an original image array of size (100, 100, 4)
original_image = img

# Define the size of the target small arrays
small_array_shape = (4, 61, 61)

# Calculate the number of slices in each dimension, including padding if necessary
num_slices_y = np.ceil(original_image.shape[0] / small_array_shape[1]).astype(int)
num_slices_x = np.ceil(original_image.shape[1] / small_array_shape[2]).astype(int)

# Pad the original image to ensure it can be evenly divided into the target slice sizes
padding_y = (num_slices_y * small_array_shape[1]) - original_image.shape[0]
padding_x = (num_slices_x * small_array_shape[2]) - original_image.shape[1]
padded_large_array = np.pad(original_image, ((0, padding_y), (0, padding_x), (0, 0)), 'constant')

# Initialize a list to hold the adjusted small arrays
small_arrays = []

# Cut the padded array into small arrays and adjust each to the desired shape
for i in range(num_slices_y):
    for j in range(num_slices_x):
        # Extract the small array
        small_array = padded_large_array[
            i*small_array_shape[1]:(i+1)*small_array_shape[1],
            j*small_array_shape[2]:(j+1)*small_array_shape[2],
            :
        ]
        # Transpose the small array to bring the channels to the front: (4, 61, 61)
        small_array_transposed = small_array.transpose(2, 0, 1)
        # Extend to (1, 1, 4, 61, 61)
        small_array_extended_corrected = np.expand_dims(small_array_transposed, axis=0)
        small_array_extended_corrected = np.expand_dims(small_array_extended_corrected, axis=0)
        small_arrays.append(small_array_extended_corrected)

# Optionally, verify the shape of the first corrected small array for confirmation
print("Shape of the first adjusted small array:", len(small_arrays))


model_outputs = []
with torch.no_grad():
    model = model.cuda()

    for i in range(len(small_arrays)):
        input = torch.FloatTensor(small_arrays[i]).cuda()
        
        output = model(input).argmax(dim=1)
        model_outputs.append(output.cpu().numpy())
        if i % 20 == 0:
            print(i)

Shape of the first adjusted small array: 5313
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660
3680
3700
3720
3740
3760
3780
3800
3820
3840
3860
3880
3900
3920
3940
3960
3980
4000
4020


In [14]:


# Initialize an empty array to merge the model outputs back into, matching the padded input array's size
merged_output_height = num_slices_y * small_array_shape[1]
merged_output_width = num_slices_x * small_array_shape[2]
merged_model_output = np.zeros((merged_output_height, merged_output_width))

# Merge each model output back into the correct position in the larger array
for i in range(num_slices_y):
    for j in range(num_slices_x):
        start_y = i * small_array_shape[1]
        start_x = j * small_array_shape[2]
        # Extract the model output slice, removing the extra dimension
        # Assuming model_outputs stores numpy arrays; if tensors, use tensor.cpu().detach().numpy() to convert
        slice_output = model_outputs[i*num_slices_x + j].squeeze()  # Shape: (61, 61)
        # Place the slice in the correct position of the merged array
        merged_model_output[start_y:start_y + small_array_shape[1], start_x:start_x + small_array_shape[2]] = slice_output

# If the original image needs to be without the added padding, you can trim the merged_model_output here
# For now, merged_model_output matches the padded dimensions

merged_model_output.shape


(4697, 4209)

In [ ]:
type(merged_model_output)
print(merged_model_output)

numpy.ndarray

In [19]:
with open("/mnt/disk1/tritd/CropFromWeb/label_snapshot-2024-03-05T00_00_00Z.npy","wb") as f:
    np.save(f, merged_model_output)

In [22]:
import numpy as np
import tifffile

# Step 1: Read the original TIFF file
with tifffile.TiffFile('/mnt/disk1/tritd/CropFromWeb/snapshot-2024-03-05T00_00_00Z.tif') as tif:
    original_data = tif.asarray()
    original_metadata = tif.pages[0].tags

# Step 2: Assuming your label array is already created and named `label_array`
# label_array = your_label_creation_function(original_data)

# Step 3 & 4: Copy metadata and modify or add additional metadata as needed
metadata_to_keep = {k: v.value for k, v in original_metadata.items() if k in ['ImageDescription', 'DateTime', 'Software']}
# Example of adding or modifying metadata
metadata_to_keep['ImageDescription'] = 'This is a label file derived from the original image'

# Step 5: Save the new TIFF file with the label data and copied/modified metadata
with tifffile.TiffWriter('/mnt/disk1/tritd/CropFromWeb/label_snapshot-2024-03-05T00_00_00Z.tif') as tif_writer:
    tif_writer.write(merged_model_output, metadata=metadata_to_keep)
